# $g$-vulnerability

Remember Evil-Eye Henry and his buried treasure from [Secrets And Vulnerability](https://github.com/damik3/qif-notebooks/blob/master/secrets_and_vulnerability.ipynb). There we discussed ways to measure the vunlnerability of a secret. Here we are gonna discuss g-vulnerability in the same context. 

Let's define $\pi$ same as before.

In [1]:
import numpy as np
try:
    from qif import *
except: # install qif if not available (for running in colab, etc)
    import IPython; IPython.get_ipython().run_line_magic('pip', 'install qif')
    from qif import *

In [2]:
pi = [1/4, 1/4, 1/8, 1/8, 1/8, 1/8]
print(pi)

[0.25, 0.25, 0.125, 0.125, 0.125, 0.125]


### Defining $g$

Remember that $g(w, x)$ specifies the gain that the adversary achieves by taking action $w$ when the value of the secret is $x$. In this example the role of the adversary is basically us trying to guess the secret location $X$.

We must first define the set $W$ which consists of the actions we can take. If we say that an action is to go and dig up a possible location, then we could define $W = \{ 1, 2, 3, 4, 5, 6 \}$. For example $w=3$ corresponds to the action of going and digging up location $3$.

When we take a guess and it is the right one, we get rewarded! In our example the reward is the monetary value of the treasure. But if we take a guess and it's the wrong one, then we lose the money we spent traveling back and forth and digging up the place. So that can be expressed with a negative number.

So $g$ could be defined as:

$$
\begin{array}{|c|c|c|c|c|c|c|}
\hline
g & \text{True} X = 1 & \text{True} X = 2 & \text{True} X = 3 & \text{True} X = 4 & \text{True} X = 5 & \text{True} X = 6 \\ \hline
\text{Guess } X = 1 & \mathbf{$1000} & -$400 & -$400 & -$400 & -$400 & -$400  \\ \hline
\text{Guess } X = 2 & -$800 & \mathbf{$1000} & -$800 & -$800 & -$800 & -$800 \\ \hline
\text{Guess } X = 3 & -$100 & -$100 & \mathbf{$1000} & -$100 & -$100 & -$100 \\ \hline
\text{Guess } X = 4 & -$200 & -$200 & -$200 & \mathbf{$1000} & -$200 & -$200 \\ \hline
\text{Guess } X = 5 & -$300 & -$300 & -$300 & -$300 & \mathbf{$1000} & -$300 \\ \hline
\text{Guess } X = 6 & -$400 & -$400 & -$400 & -$400 & -$400 & \mathbf{$1000} \\ \hline
\end{array}
$$

ok

$$
\text{Guess } X = 1 \mathbf{$1000} -$400 -$400 -$400 -$400 -$400  \\
$$

okok

$$
\mathbf{$1000} -$400 -$400 -$400 -$400 -$400  \\
$$

okokok

$$
\text{Guess } X = 1 -$400 -$400 -$400 -$400 -$400  \\
$$

$g$'s first line corresponds to choosing to dig up location 1. 

 - $g(1, 1)$ means we choose 1 and the treasure is indeed there. So we get our prize of $ \$ 1000$! 
 - $g(1, 2)$ means we choose 1 and the treasure is in 2. So we spend $1000$ which are the digging expenses for location 2. 
 - $g(1, 3)$ means we choose 1 and the treasure is in 3. Again we spend $1000$ because we still chose to dig in location 2. 
 - ...
 
The same logic applies to the rest of $g$.

In [3]:
g = np.array([
    [1000, -400, -400, -400, -400, -400],
    [-800, 1000, -750, -750, -750, -750],
    [-100, -100, 1000, -100, -100, -100],
    [-200, -200, -200, 1000, -200, -200],
    [-300, -300, -300, -300, 1000, -300],
    [-400, -400, -400, -400, -400, 1000],
])
print(g)

[[1000 -400 -400 -400 -400 -400]
 [-800 1000 -750 -750 -750 -750]
 [-100 -100 1000 -100 -100 -100]
 [-200 -200 -200 1000 -200 -200]
 [-300 -300 -300 -300 1000 -300]
 [-400 -400 -400 -400 -400 1000]]


### Calculating g-vulnerability

For each action we take we cannot be sure about what we gain from it. That dependes on the true value of $X$. But we can make an estimate. Based on the probability distribution of $X$. We can compute for each action the average gain we obtain. 

In [4]:
exp_gain = np.matmul(g, np.transpose(pi))
for i in range(len(exp_gain)):
    print("Average earnings for choosing location %d: %.2f" % (i+1, exp_gain[i]))

Average earnings for choosing location 1: -50.00
Average earnings for choosing location 2: -325.00
Average earnings for choosing location 3: 37.50
Average earnings for choosing location 4: -50.00
Average earnings for choosing location 5: -137.50
Average earnings for choosing location 6: -225.00


And now what would our best choice be? The one with the highest average winnings of course!

In [5]:
print("Best choice: Location", np.argmax(exp_gain)+1)
print("Expected winnings: $", max(exp_gain), sep='')

Best choice: Location 3
Expected winnings: $37.5


Location 1 has a higher probability of being the true value of $X$ but it costs us more if we are wrong. On the other hand, location 3 has a lower probability of being the true value of $X$ but it costs us less if we are wrong. But that lower probability balances out with the smaller cost for when being wrong. And on average, makes a better choice than location 1.

Notice also that without considering $g$, our best choice would have been to guess $X=1$. But given the information $g$ provides us, $X=3$ is a better guess.